 # Signal analysis
 
 Analyze the performance of beamforming with different filtering schemes, such as snr-based filtering, props filtering, etc. 

In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from evaluate_data import read_df
from crazyflie_description_py.parameters import N_BUFFER, FS, FFTSIZE
import itertools

exp_name = '2020_11_03_sweep'; start_idx = 20

all_frequencies = np.round(np.fft.rfftfreq(N_BUFFER, 1/FS), 0)
#print(list(all_frequencies))

df_results = pd.DataFrame(columns=[
    'degree', 
    'props',
    'snr',
    'motors',
    'source',
    'times',
    'spectrogram',
    'frequency_matrix',
    'strongest_freqs'
])

DEGREE_LIST = [90]
PROPS_LIST = [False, True]
SNR_LIST = [False, True]
MOTORS_LIST = [False, True]
SOURCE_LIST = ['sweep']

for params_values in itertools.product(DEGREE_LIST, PROPS_LIST, SNR_LIST, MOTORS_LIST, SOURCE_LIST):
    params_dict = dict(zip(['degree', 'props', 'snr', 'motors', 'source'], params_values))
    params_dict['exp_name'] = exp_name

    df, df_pos = read_df(**params_dict)
    df = df.iloc[start_idx:]
    
    frequency_matrix = np.zeros((len(all_frequencies), len(df)))
    i_col = 0
    for j, row in df.iterrows():
        # check if frequencies are sorted
        #if not np.all(np.diff(row.frequencies) >= 0):
            #print('not sorted:', params_dict)
        for f in row.frequencies:
            
            # find the closest frequency bin to chosen frequency
            idx = np.argmin(np.abs(f-all_frequencies))
            err = abs(all_frequencies[idx] - f)
            #if err >= 1e-1:
                #print(f'did not find {f} in bins. error:', err)
            frequency_matrix[idx, i_col] = 1.0 
        sum_ = np.sum(frequency_matrix[:, i_col])
        assert sum_ == FFTSIZE, sum_
        i_col += 1
        
    spectrogram = np.zeros((FFTSIZE, len(df)))
    strongest_freqs = np.zeros(len(df))
    i_col = 0
    for __, row in df.iterrows():
        signals_f = row.signals_f
        
        spectrogram[:, i_col] = np.sum(np.abs(signals_f), axis=0)
        
        max_idx = np.argmax(spectrogram[:, i_col])
        #if params_dict['snr']:
        #    assert max_idx == 0, max_idx
        strongest_freqs[i_col] = row.frequencies[max_idx]
        i_col += 1
    
    params_dict['spectrogram'] = spectrogram
    params_dict['strongest_freqs'] = strongest_freqs
    params_dict['frequency_matrix'] = frequency_matrix
    params_dict['times'] = (df.loc[:, 'audio_timestamp'].values - df.iloc[0].audio_timestamp) / 1e6
    df_results.loc[len(df_results), :] = params_dict
df_results.tail()

In [ ]:
shape = len(SNR_LIST)*len(PROPS_LIST), len(MOTORS_LIST)
fig, axs = plt.subplots(*shape, sharey=True, sharex=True)
fig.set_size_inches(shape[0]*5, shape[1]*5)

fig_spec, axs_spec = plt.subplots(*shape, sharey=True, sharex=True)
fig_spec.set_size_inches(shape[0]*5, shape[1]*5)

fig_scat, axs_scat = plt.subplots(*shape, sharex=True, sharey=True)
fig_scat.set_size_inches(shape[0]*5, shape[1]*5)
for j, motors in enumerate(MOTORS_LIST):
    axs[0, j].set_title(f"motors {motors}")
    axs_scat[0, j].set_title(f"motors {motors}")
    for i, (snr, props) in enumerate(itertools.product(SNR_LIST, PROPS_LIST)):
        row = df_results.loc[(df_results.motors==motors) & (df_results.snr==snr) & (df_results.props==props), :]
        row = row.iloc[0]
        spectrogram = row.spectrogram
        spectrogram_wide = np.full((len(all_frequencies), spectrogram.shape[1]), np.nan)
        mask = np.where(row.frequency_matrix > 0)
        spectrogram_wide[mask[0], mask[1]] = spectrogram.flatten()
        
        times = row.times
        
        plot_mask = (all_frequencies < 7000) & (all_frequencies > 200)
        
        axs[i, j].pcolormesh(times, all_frequencies[plot_mask], np.log(spectrogram_wide[plot_mask]))
        axs[i, 0].set_ylabel(f"snr {snr} props {props}")
        
        axs_spec[i, j].pcolormesh(times, range(FFTSIZE), np.log(spectrogram))
        
        axs_scat[i, j].scatter(times, row.strongest_freqs)
        
        axs_spec[i, j].set_ylabel(f"snr {snr} props {props}")

In [ ]:
from audio_stack.beam_former import BeamFormer, combine_rows, normalize_rows
import progressbar

DEGREE_LIST = [90]
PROPS_LIST = [False, True]
SNR_LIST = [False, True]
MOTORS_LIST = [False, True]
SOURCE_LIST = ['sweep']

METHOD_LIST = ['mvdr', 'das']
combination_method = 'sum'

#INVERSE = 'pinv'
#fname = f'results/SignalAnalysis_{exp_name}_pinv.pkl'

INVERSE = 'low-rank'
fname = f'results/SignalAnalysis_{exp_name}_low-rank.pkl'

try:
    df_spectra = pd.read_pickle(fname)
    print('read', fname)
except:
    df_spectra = pd.DataFrame(columns=[
        'degree', 
        'props',
        'snr',
        'motors',
        'source',
        'method',
        'times',
        'raw_heatmap',
    ])

    for params_values in itertools.product(DEGREE_LIST, PROPS_LIST, SNR_LIST, MOTORS_LIST, SOURCE_LIST):
        params_dict = dict(zip(['degree', 'props', 'snr', 'motors', 'source'], params_values))
        params_dict['exp_name'] = exp_name

        df, df_pos = read_df(**params_dict)
        df = df.iloc[start_idx:]
        n_columns = len(df)
        times = (df.loc[:, 'audio_timestamp'].values - df.iloc[0].audio_timestamp) / 1e6
        mic_positions = df.iloc[0].mic_positions

        for method, in itertools.product(METHOD_LIST):
            print(method)
            params_dict['method'] = method

            beam_former = BeamFormer(mic_positions=mic_positions)
            angles = beam_former.theta_scan
            raw_heatmap = np.zeros((len(angles), n_columns))

            i_col = 0
            with progressbar.ProgressBar(max_value=n_columns, redirect_stdout=True) as p:
                for __, row in df.iterrows():

                    # choose strongest frequency
                    spectrogram = np.sum(np.abs(row.signals_f), axis=0)
                    selected_idx = [np.argmax(spectrogram)]

                    signals_f = row.signals_f[:, selected_idx] # n_mics x n_freqs
                    freqs = row.frequencies[selected_idx]

                    R = beam_former.get_correlation(signals_f.T)
                    if method == 'mvdr':
                        raw_spectrum = beam_former.get_mvdr_spectrum(R, freqs, inverse=INVERSE)
                    elif method == 'das':
                        raw_spectrum = beam_former.get_das_spectrum(R, freqs)
                    else:
                        raise ValueError(method)

                    raw_spectrum = combine_rows(raw_spectrum, combination_method, keepdims=True)
                    raw_spectrum = normalize_rows(raw_spectrum, method="zero_to_one")
                    raw_heatmap[:, i_col] = raw_spectrum.flatten()

                    i_col += 1
                    p.update(i_col)
                    if i_col >= n_columns:
                        break

                params_dict['raw_heatmap'] = raw_heatmap
                params_dict['times'] = times 
                df_spectra.loc[len(df_spectra), :] = params_dict
    df_spectra.to_pickle(fname)
    print('saved as', fname)
df_spectra.tail()

In [ ]:
#angles = BeamFormer.theta_scan
angles = np.linspace(0, 360, df_spectra.iloc[0].raw_heatmap.shape[0])

# TODO(FD): read these from parameters file
min_freq = 200
max_freq = 7000

for method in METHOD_LIST:

    fig_spec, axs_spec = plt.subplots(*shape, sharey=True, sharex=True)
    fig_spec.set_size_inches(shape[0]*5, shape[1]*5)

    for j, motors in enumerate(MOTORS_LIST):
        axs[0, j].set_title(f"motors {motors}")
        axs_scat[0, j].set_title(f"motors {motors}")
        for i, (snr, props) in enumerate(itertools.product(SNR_LIST, PROPS_LIST)):
            row = df_spectra.loc[
                (df_spectra.motors==motors) & 
                (df_spectra.snr==snr) & 
                (df_spectra.props==props) & 
                (df_spectra.method==method), :]
            row = row.iloc[0]
            heatmap = row.raw_heatmap

            times = row.times
            plot_mask = (all_frequencies < max_freq) & (all_frequencies > min_freq)

            axs_spec[i, j].pcolormesh(times, angles, heatmap)
            axs_spec[i, 0].set_ylabel(f"snr {snr} props {props}")
            axs_spec[i, 1].set_ylabel(f"snr {snr} props {props}")

# Sandbox

In [ ]:
from algos_basics import low_rank_inverse

def low_rank_tensor_inverse(low_rank_tensor, rank=1):
    # low_rank_tensor is of shape n x m x m, and we take the inverse
    # of each m x m slice. 
    u, s, vt = np.linalg.svd(low_rank_tensor, full_matrices=True)
    
    # TODO(FD) could be done more efficiently with broadcasting? 
    s_mat = np.concatenate([[np.zeros((s.shape[1], s.shape[1]))]]*s.shape[0])
    s_mat[:, range(rank), range(rank)] = 1 / s[..., :rank]
    a = np.transpose(vt, (0, 2, 1))
    b = np.transpose(u, (0, 2, 1))
    
    # TODO(FD understand why below we need to take element 0)
    inverse = a.dot(s_mat).dot(b)[:, :, 0, 0, :]
    #inverse = np.matmul(a, s_mat, b)
    return inverse

n_mics = 4
lamda = 1e-3
#signals_f = np.arange(1, n_mics+1).reshape((n_mics, 1))
signals_f = np.random.rand(1, n_mics).reshape((n_mics, 1))
signals_f_ = np.random.rand(1, n_mics).reshape((n_mics, 1))
rank_1 = signals_f.dot(signals_f.T) 
rank_1_ = signals_f_.dot(signals_f_.T) 
rank_1_tensor = np.concatenate([rank_1[None, :, :], rank_1_[None, :, :]], axis=0)

own_tensor = low_rank_tensor_inverse(rank_1_tensor)
own = low_rank_inverse(rank_1 + lamda*np.eye(n_mics))
own_ = low_rank_inverse(rank_1_)

print('tensor', own_tensor)
print('own', own.dot(rank_1))
print('own_', own_.dot(rank_1_))

pinv = np.linalg.pinv(rank_1)
inv = np.linalg.inv(rank_1 + lamda*np.eye(n_mics))
print(pinv.dot(rank_1))
print(inv.dot(rank_1))